### Библиотеки / данные

импортируем библиотеки numpy и pandas

In [1]:
import numpy as np
import pandas as pd

задаем некоторые настройки pandas, регулирующие формат вывода

In [2]:
pd.options.display.max_rows = 10

загружаем данные

In [3]:
tips = pd.read_csv("tips.csv")
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [4]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [5]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


### GroupBy

In [6]:
df = pd.DataFrame({'x': ['a','a','b','b','c','c'],
                   'y': [2,4,0,5,5,10]})
df

,x,y
0,a,2
1,a,4
2,b,0
3,b,5
4,c,5
5,c,10


In [7]:
groups = df.groupby(['x'])
groups

получаем информацию о количестве групп, которые будут созданы

In [8]:
groups.ngroups

3

получаем информацию о количестве элементов в каждой группе

In [9]:
groups.size()

x
a    2
b    2
c    2
dtype: int64

что представляют из себя найденные группы?

In [10]:
groups.groups

{'a': [0, 1], 'b': [2, 3], 'c': [4, 5]}

получаем данные конкретной группы

In [11]:
groups.get_group('b')

,x,y
2,b,0
3,b,5


извлекаем первую строку каждой группы

In [12]:
groups.nth([1])

,y
x,
a,4
b,5
c,10


обход групп:

In [13]:
for key, group in groups:
    print(key)
    print(group) 

a
   x  y
0  a  2
1  a  4
b
   x  y
2  b  0
3  b  5
c
   x   y
4  c   5
5  c  10


вычисление среднего

In [14]:
groups.y.mean()

x
a    3.0
b    2.5
c    7.5
Name: y, dtype: float64

### Типы группировок

#### по столбцам: 

In [15]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [16]:
tips.groupby(['day','time']).tip.mean().unstack()

time,Dinner,Lunch
day,,
Fri,2.940000,2.382857
Sat,2.993103,NaN
Sun,3.255132,NaN
Thur,3.000000,2.767705


#### по уровням индекса

создаем копию данных и заново индексируем ее

In [17]:
copy_tips = tips.copy()
copy_tips = copy_tips.set_index(['day', 'time'])
copy_tips

total_bill   tip smoker  size   tip_pct
day  time                                           
Sun  Dinner       16.99  1.01     No     2  0.059447
     Dinner       10.34  1.66     No     3  0.160542
     Dinner       21.01  3.50     No     3  0.166587
     Dinner       23.68  3.31     No     2  0.139780
     Dinner       24.59  3.61     No     4  0.146808
...                 ...   ...    ...   ...       ...
Sat  Dinner       29.03  5.92     No     3  0.203927
     Dinner       27.18  2.00    Yes     2  0.073584
     Dinner       22.67  2.00    Yes     2  0.088222
     Dinner       17.82  1.75     No     2  0.098204
Thur Dinner       18.78  3.00     No     2  0.159744

[244 rows x 5 columns]

группировать можем по одному или нескольким уровням индекса, передавая соответствующие значения столбцов аргументу level

In [18]:
copy_tips.groupby(level=['time']).sum()

,total_bill,tip,size,tip_pct
time,,,,
Dinner,3660.30,546.07,463,28.075131
Lunch,1167.47,185.51,164,11.160699


In [19]:
copy_tips.groupby(level = ['day', 'time']).mean()

total_bill       tip      size   tip_pct
day  time                                            
Fri  Dinner   19.663333  2.940000  2.166667  0.158916
     Lunch    12.845714  2.382857  2.000000  0.188765
Sat  Dinner   20.441379  2.993103  2.517241  0.153152
Sun  Dinner   21.410000  3.255132  2.842105  0.166897
Thur Dinner   18.780000  3.000000  2.000000  0.159744
     Lunch    17.664754  2.767705  2.459016  0.161301

#### с использованием функции 

In [20]:
copy_tips.head()

total_bill   tip smoker  size   tip_pct
day time                                           
Sun Dinner       16.99  1.01     No     2  0.059447
    Dinner       10.34  1.66     No     3  0.160542
    Dinner       21.01  3.50     No     3  0.166587
    Dinner       23.68  3.31     No     2  0.139780
    Dinner       24.59  3.61     No     4  0.146808

In [21]:
copy_tips.groupby(len, level='day').count()

,total_bill,tip,smoker,size,tip_pct
day,,,,,
3,182,182,182,182,182
4,62,62,62,62,62


In [22]:
tips.day.value_counts()

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

#### с использованием массива 

массив случайных меток групп:

In [23]:
np.random.seed(123)
rnd_array = np.random.choice(['first_group', 'second_group'], 
                             size=tips.shape[0],
                             p = [0.4, 0.6])
rnd_array[0:5]

array(['second_group', 'first_group', 'first_group', 'second_group',
       'second_group'], dtype='<U12')

передаем массив меток групп в метод groupby

In [24]:
tips.groupby(rnd_array).count()

,total_bill,tip,smoker,day,time,size,tip_pct
first_group,93,93,93,93,93,93,93
second_group,151,151,151,151,151,151,151


In [25]:
93/(93+151)

0.38114754098360654

#### бонус

тип данных:

In [26]:
tips.dtypes

total_bill    float64
tip           float64
smoker         object
day            object
time           object
size            int64
tip_pct       float64
dtype: object

In [27]:
group_tips = tips.groupby(tips.dtypes, axis=1)

In [28]:
group_tips.size()

int64      1
float64    3
object     3
dtype: int64

### Агрегирование групп

создаем объект groupby:

In [29]:
group_tips = tips.groupby(['day', 'smoker'])['tip_pct']

проверка эквивалентности методов:

In [30]:
group_tips.agg == group_tips.aggregate

True

собственная функция:

In [31]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

group_tips.agg([peak_to_peak])

peak_to_peak
day  smoker              
Fri  No          0.067349
     Yes         0.159925
Sat  No          0.235193
     Yes         0.290095
Sun  No          0.193226
     Yes         0.644685
Thur No          0.193350
     Yes         0.151240

комбинирование:

In [32]:
group_tips.agg([peak_to_peak, 'mean'])

peak_to_peak      mean
day  smoker                        
Fri  No          0.067349  0.151650
     Yes         0.159925  0.174783
Sat  No          0.235193  0.158048
     Yes         0.290095  0.147906
Sun  No          0.193226  0.160113
     Yes         0.644685  0.187250
Thur No          0.193350  0.160298
     Yes         0.151240  0.163863

задаем имена столбцов:

In [33]:
group_tips.agg([('delta_max_min', peak_to_peak), ('mean_value','mean')])

delta_max_min  mean_value
day  smoker                           
Fri  No           0.067349    0.151650
     Yes          0.159925    0.174783
Sat  No           0.235193    0.158048
     Yes          0.290095    0.147906
Sun  No           0.193226    0.160113
     Yes          0.644685    0.187250
Thur No           0.193350    0.160298
     Yes          0.151240    0.163863

отдельные наборы функций для каждого столбца:

In [34]:
group_tips = tips.groupby(['day', 'smoker'])['tip_pct', 'total_bill']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [35]:
group_tips.agg({'tip_pct':[('max_value', np.max),
                           ('min_value','min')],
                'total_bill': 'sum'})

tip_pct           total_bill
            max_value min_value        sum
day  smoker                               
Fri  No      0.187735  0.120385      73.68
     Yes     0.263480  0.103555     252.20
Sat  No      0.291990  0.056797     884.78
     Yes     0.325733  0.035638     893.62
Sun  No      0.252672  0.059447    1168.88
     Yes     0.710345  0.065660     458.28
Thur No      0.266312  0.072961     770.09
     Yes     0.241255  0.090014     326.24

### Преобразование групп

#### метод transform

In [36]:
df = pd.DataFrame({'Col1': ['A', 'B', 'C', 'C', 'B', 'B', 'A'],
                   'Col2': [1, 2, 3, 4, 2, 5, 3]})
df

,Col1,Col2
0,A,1
1,B,2
2,C,3
3,C,4
4,B,2
5,B,5
6,A,3


In [37]:
df['Col3'] = df.groupby('Col1').transform(sum)
df.sort_values('Col1')

,Col1,Col2,Col3
0,A,1,4
6,A,3,4
1,B,2,9
4,B,2,9
5,B,5,9
2,C,3,7
3,C,4,7


объект groupby:

In [38]:
group_tips = tips.groupby(['smoker'])['total_bill']

нормирование:

In [39]:
norm = lambda x: (x - x.mean())/x.std()
group_tips.transform(norm)

0     -0.266278
1     -1.071793
2      0.220665
3      0.544083
4      0.654311
         ...   
239    1.192129
240    0.653331
241    0.194632
242   -0.165740
243   -0.049455
Name: total_bill, Length: 244, dtype: float64

In [40]:
tips_copy = tips.copy()
tips_copy['total_bill_norm'] = group_tips.transform(norm)
tips_copy.head()

,total_bill,tip,smoker,day,time,size,tip_pct,total_bill_norm
0,16.99,1.01,No,Sun,Dinner,2,0.059447,-0.266278
1,10.34,1.66,No,Sun,Dinner,3,0.160542,-1.071793
2,21.01,3.50,No,Sun,Dinner,3,0.166587,0.220665
3,23.68,3.31,No,Sun,Dinner,2,0.139780,0.544083
4,24.59,3.61,No,Sun,Dinner,4,0.146808,0.654311


In [41]:
tips_copy.groupby('smoker').total_bill_norm.agg(['mean', 'std'])

,mean,std
smoker,,
No,-8.156187e-16,1.0
Yes,-1.635490e-16,1.0


#### метод apply

функция:

In [42]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


применяем метод apply

In [43]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

### Исключение групп

создаем данные для наших примеров

In [44]:
df = pd.DataFrame({'Label': list('AABCCC'),
                   'Values': [1, 2, 3, 4, np.nan, 8]})
df

,Label,Values
0,A,1.0
1,A,2.0
2,B,3.0
3,C,4.0
4,C,NaN
5,C,8.0


удаляем группы с менее чем двумя возможными значениеми

In [45]:
f = lambda x: x.Values.count() > 1
df.groupby('Label').filter(f)

,Label,Values
0,A,1.0
1,A,2.0
3,C,4.0
4,C,NaN
5,C,8.0


удаляем группы, в которых есть пропуски

In [46]:
f = lambda x: x.Values.isnull().sum() == 0
df.groupby('Label').filter(f)

,Label,Values
0,A,1.0
1,A,2.0
2,B,3.0


### Сводная таблица

средние

In [47]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

вывод по строкам и столбцам

In [48]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

включение частичных итогов:

In [49]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

задаем функцию:

In [50]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                 aggfunc=max, margins=True)

day                 Fri       Sat       Sun      Thur       All
time   smoker                                                  
Dinner No      0.155625  0.291990  0.252672  0.159744  0.291990
       Yes     0.263480  0.325733  0.710345       NaN  0.710345
Lunch  No      0.187735       NaN       NaN  0.266312  0.266312
       Yes     0.259314       NaN       NaN  0.241255  0.259314
All            0.263480  0.325733  0.710345  0.266312  0.710345